In [10]:
import os
import csv, json

from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown

load_dotenv("backend/.env")
API_KEY = os.getenv("GEMINI_API_KEY")
MODEL = "gemini-2.5-flash"

client = OpenAI(
    api_key=API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

# Tools

In [2]:
from datetime import datetime
from typing import List, Dict

def get_weekday_names(dates: List[str]) -> Dict[str, str]:
    """Gets the weekday name for each date.
    
    Args:
        dates (List[str]): List of dates to check in YYYY-MM-DD format.
        
    Returns:
        Dict[str, str]: Dictionary mapping each date to its weekday name (Monday, Tuesday, etc.).
    """
    result = {}
    for date in dates:
        weekday = datetime.strptime(date, "%Y-%m-%d").strftime("%A")
        result[date] = weekday
    return result


#############################################
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weekday_names",
            "description": "Gets the weekday name for each date.",
            "parameters": {
                "type": "object",
                "properties": {
                    "dates": {
                        "type": "array",
                        "items": {
                            "type": "string"
                        },
                        "description": "List of dates to check in YYYY-MM-DD format."
                    }
                },
                "required": ["dates"]
            }
        }
    }
]

tools

[{'type': 'function',
  'function': {'name': 'get_weekday_names',
   'description': 'Gets the weekday name for each date.',
   'parameters': {'type': 'object',
    'properties': {'dates': {'type': 'array',
      'items': {'type': 'string'},
      'description': 'List of dates to check in YYYY-MM-DD format.'}},
    'required': ['dates']}}}]

# Production Order Test

In [4]:
use_json = True

if use_json:
    with open("data/production_plan_1.csv", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        data = list(reader)

    data = json.dumps(data)
else:
    data = open("data/production_plan_1.csv", encoding="utf-8").read()

CURRENT_STOCK = 2
SCRAP_RATE = 0.05

data


'[{"date": "2025-05-28", "transaction_type": "production_order", "product": "Product A", "quantity": "96"}, {"date": "2025-05-30", "transaction_type": "production_order", "product": "Product A", "quantity": "96"}, {"date": "2025-06-09", "transaction_type": "production_order", "product": "Product A", "quantity": "96"}, {"date": "2025-06-10", "transaction_type": "production_order", "product": "Product A", "quantity": "96"}, {"date": "2025-06-10", "transaction_type": "production_order", "product": "Product A", "quantity": "96"}, {"date": "2025-06-11", "transaction_type": "production_order", "product": "Product A", "quantity": "96"}, {"date": "2025-06-30", "transaction_type": "sales_order", "product": "Product A", "quantity": "-96"}, {"date": "2025-06-30", "transaction_type": "sales_order", "product": "Product A", "quantity": "-96"}, {"date": "2025-07-07", "transaction_type": "sales_order", "product": "Product A", "quantity": "-96"}, {"date": "2025-07-07", "transaction_type": "sales_order"

In [11]:
MODEL_INPUT = f"""
Please analyze the following data and output an optimized production plan:
{data}

The current stock level is {CURRENT_STOCK} units of Product A.
The scrap rate is {SCRAP_RATE*100}%.
"""

SYSTEM_INSTRUCTION = """
# Role
You are an expert in the field of manufacturing and supply chain management.
You analyze the production and sales orders of a company, aiming to optimize their scheduling.
Sales orders are fixed, but you can change both the quantity and the date of production orders.
Sales orders are negative, production orders are positive.
You are concise and to the point.

# Background information
- The maximum capacity of the factory is 100 units of product A per day. This cannot be exceeded at any time.
- Note that the product needs to be produced one day before it is sold (used in a sales order).
- Take the scrap rate into account. This means that you need to produce more than the sales order quantity to account for the scrap.
- No production is allowed on weekends (Saturday and Sunday).

# Objective
You optimize the production plan to optimize the following KPIs:
- OTIF (On Time In Full): Aim to never miss a sales order due date. This is the most important KPI. Stockouts should be avoided if at all possible.
- At the same time, aim to minimize the number of days between production and sales to reduce finished goods inventory (ideally, ensure Just-In-Time production).

# Output
You first output should always be a table in valid Markdown format. The table includes the rescheduled, optimized production orders as well as the sales orders (which you may not change).
The following columns are required:
- Date
- Product
- Type (🛠️ Production Order or 💰 Sales Order)
- Quantity (negative, if a sales order, positive if a production order)

Below that, break down the calculations and very briefly explain assumptions and reasoning.
"""

conversation_history = [{"role": "system", "content": SYSTEM_INSTRUCTION}]
conversation_history.extend([{"role": "user", "content": MODEL_INPUT}])

# Function to handle tool calls
def execute_tool_call(tool_call):
    # Extract function name and arguments from the tool call
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    
    # Execute the appropriate function based on name
    if function_name == "get_weekday_names":
        return get_weekday_names(**function_args)
    else:
        return f"Unknown function: {function_name}"

# Initial API call
response = client.chat.completions.create(
    model=MODEL,
    stream=False,
    messages=conversation_history,
    tools=tools
)

# Process the response and handle any tool calls
message = response.choices[0].message
final_response = None

# Add the initial response to conversation history
# Note: We need to be careful with tool_calls which might be None
assistant_message = {"role": "assistant", "content": message.content}
if hasattr(message, 'tool_calls') and message.tool_calls:
    assistant_message["tool_calls"] = message.tool_calls
conversation_history.append(assistant_message)

# Process tool calls if they exist
max_iterations = 5  # Limit iterations to avoid infinite loops
iteration = 0

while hasattr(message, 'tool_calls') and message.tool_calls:
    iteration += 1
    print(f"Processing tool calls iteration {iteration}")
    
    # Process each tool call
    for tool_call in message.tool_calls:
        # Execute the tool call
        result = execute_tool_call(tool_call)
        print(f"Tool call: {tool_call.function.name} -> Result: {result} - full: {tool_call}")
        
        # Add the tool call result to the conversation
        conversation_history.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_call.function.name,
            "content": str(result)
        })
    
    # Make another API call with the updated conversation
    response = client.chat.completions.create(
        model=MODEL,
        stream=False,
        messages=conversation_history,
        tools=tools
    )
    
    # Update the message for the next iteration
    message = response.choices[0].message
    
    # Add the assistant's response to the conversation
    assistant_message = {"role": "assistant", "content": message.content}
    if hasattr(message, 'tool_calls') and message.tool_calls:
        assistant_message["tool_calls"] = message.tool_calls
    conversation_history.append(assistant_message)
    
    # If no more tool calls, this is the final response
    if not hasattr(message, 'tool_calls') or not message.tool_calls:
        final_response = message.content
        print(f"No more tool calls, finishing after {iteration} iterations")

    if iteration > max_iterations:
        print(f"Max iterations reached after {iteration} iterations")
        break

# Print the final response
final_response = final_response if final_response else message.content
Markdown(final_response)

Processing tool calls iteration 1
Tool call: get_weekday_names -> Result: {'2025-06-26': 'Thursday', '2025-06-27': 'Friday', '2025-06-28': 'Saturday', '2025-06-29': 'Sunday', '2025-06-30': 'Monday', '2025-07-04': 'Friday', '2025-07-05': 'Saturday', '2025-07-06': 'Sunday', '2025-07-07': 'Monday', '2025-07-11': 'Friday', '2025-07-12': 'Saturday', '2025-07-13': 'Sunday', '2025-07-14': 'Monday'} - full: ChatCompletionMessageToolCall(id='', function=Function(arguments='{"dates":["2025-06-26","2025-06-27","2025-06-28","2025-06-29","2025-06-30","2025-07-04","2025-07-05","2025-07-06","2025-07-07","2025-07-11","2025-07-12","2025-07-13","2025-07-14"]}', name='get_weekday_names'), type='function')
No more tool calls, finishing after 1 iterations


| Date | Product | Type | Quantity |
|---|---|---|---|
| 2025-06-26 | Product A | 🛠️ Production Order | 101 |
| 2025-06-27 | Product A | 🛠️ Production Order | 101 |
| 2025-06-30 | Product A | 💰 Sales Order | -96 |
| 2025-06-30 | Product A | 💰 Sales Order | -96 |
| 2025-07-04 | Product A | 🛠️ Production Order | 101 |
| 2025-07-07 | Product A | 💰 Sales Order | -96 |
| 2025-07-07 | Product A | 💰 Sales Order | -96 |
| 2025-07-11 | Product A | 🛠️ Production Order | 101 |
| 2025-07-14 | Product A | 💰 Sales Order | -96 |

Calculations and Reasoning:

- Sales orders remain unchanged as per the instructions.
- The scrap rate of 5.0% means that for every 96 units sold, 96 / (1 - 0.05) = 101 units must be produced.
- Production orders are scheduled one day prior to the sales order to ensure on-time delivery while minimizing inventory.
- Production is limited to 100 units per day and is not scheduled on weekends.
- Existing production orders were disregarded as the objective is to create an optimized plan.
- The initial stock of 2 units has been incorporated into the plan.

In [12]:
# Debug cell to print the conversation history
print("Conversation History:")
for i, msg in enumerate(conversation_history):
    print(f"\n[{i}] Role: {msg['role']}")
    
    # Print content (truncate if too long)
    content = msg.get('content', 'None')
    if content and len(content) > 200:
        print(f"Content: {content[:200]}...")
    else:
        print(f"Content: {content}")
    
    # Check if there are tool calls
    if msg['role'] == 'assistant' and msg.get('tool_calls'):
        print("Tool Calls:")
        [print(f"\t{tool_call.function}") for tool_call in msg["tool_calls"]]
    
    # Print tool name for tool responses
    if msg['role'] == 'tool':
        print(f"Tool: {msg.get('name')}")


Conversation History:

[0] Role: system
Content: 
# Role
You are an expert in the field of manufacturing and supply chain management.
You analyze the production and sales orders of a company, aiming to optimize their scheduling.
Sales orders are fix...

[1] Role: user
Content: 
Please analyze the following data and output an optimized production plan:
[{"date": "2025-05-28", "transaction_type": "production_order", "product": "Product A", "quantity": "96"}, {"date": "2025-05...

[2] Role: assistant
Content: None
Tool Calls:
	Function(arguments='{"dates":["2025-06-26","2025-06-27","2025-06-28","2025-06-29","2025-06-30","2025-07-04","2025-07-05","2025-07-06","2025-07-07","2025-07-11","2025-07-12","2025-07-13","2025-07-14"]}', name='get_weekday_names')

[3] Role: tool
Content: {'2025-06-26': 'Thursday', '2025-06-27': 'Friday', '2025-06-28': 'Saturday', '2025-06-29': 'Sunday', '2025-06-30': 'Monday', '2025-07-04': 'Friday', '2025-07-05': 'Saturday', '2025-07-06': 'Sunday', '...
Tool: get_we